In [ ]:
import pandas as pd
import numpy as np
# import prince
# from lightfm import LightFM
# from tqdm import tqdm
import seaborn as sns
import umap.umap_ as umap
import matplotlib.pyplot as plt
# import scipy.sparse as ss
from datetime import datetime
from sklearn.cluster import KMeans
from scipy.stats import chi2_contingency
import seaborn as sns

In [ ]:
from datetime import datetime
from datetime import date
date = date.today()
period = "AUG"
run = f"{date}_{period}"
run

In [ ]:
def emb_prep(emb):
    emb_short = emb[["client_id", "is_user", "hits", "hits_watches", "days_active", "cluster"]].copy()
    del emb
    emb_short["hits_per_day"] = round(emb_short["hits"]/emb_short["days_active"], 2)
    emb_short["watches_hits_ratio"] = round(emb_short["hits_watches"] / emb_short["hits"], 2)
    return emb_short


In [ ]:
# user_embed_cat_num = pd.read_parquet(f"{run}_embeddings_with_num.pq")
emb = pd.read_parquet("explore_short.pq")

In [ ]:
# features before embeddings

devices_user = pd.read_parquet("users_device_features_2023-04-20_AUG (1).pq")
tags_user = pd.read_parquet("AUG_tags_per_user.pq")
pages_user = pd.read_parquet("3m_pages_features.pq")
events_user = pd.read_parquet("3m_events_features.pq")



In [ ]:
emb = emb.drop(columns=["cluster"]).rename(columns={"cluster_6": "cluster"})
emb_short = emb_prep(emb)

In [ ]:
clusters_sizes = emb_short.groupby(["cluster"]).client_id.count().reset_index()
clusters_sizes = clusters_sizes.rename(columns={"client_id": "cluster_size"})
clusters_sizes["size_ratio"] = clusters_sizes["cluster_size"]/clusters_sizes["cluster_size"].sum()
clusters_sizes

In [ ]:
activity_by_cluster = emb_short.groupby(["cluster"]).agg({"hits": np.median,\
                                           "hits_watches": np.median,\
                                           "days_active": np.median,\
                                           "hits_per_day": np.median,\
                                           "watches_hits_ratio": np.median})

In [ ]:
activity_by_cluster

In [ ]:
totals_df = pd.DataFrame(columns=activity_by_cluster.columns)
value_list = list()
for col in activity_by_cluster.columns:
    value = np.median(emb_short[col])
    value_list.append(value)

totals_df = activity_by_cluster.copy()
totals_df = totals_df.append(pd.DataFrame([value_list], 
     columns=activity_by_cluster.columns), 
     ignore_index=True)
totals_df

In [ ]:
hits_2 = emb_short[emb_short["cluster"] == 1]['hits']
hits_7 = emb_short[emb_short["cluster"] == 4]['hits']
totals = emb_short['hits']

upper_clip = 300

sns.set_theme()
plt.figure(figsize=(12, 6), dpi=80)
_, bins, _ = plt.hist(hits_2.clip(upper=upper_clip), bins=100, alpha=0.5, label='Casual browsers', weights=np.ones(hits_2.shape[0])/hits_2.shape[0])
# plt.hist(hits_7.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Logged buyers hits', weights=np.ones(hits_7.shape[0])/hits_7.shape[0])
plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
emb_gr = emb_short.groupby(["cluster", "is_user"]).client_id.count().reset_index()

clusters_users = pd.pivot_table(emb_gr, index=["cluster"],
                                 columns=["is_user"], values="client_id", fill_value=0)\
    .reset_index()

clusters_users.columns = ["cluster", "non_user", "user"]
clusters_users["user_ratio"] = round(clusters_users["user"]/(clusters_users["user"] + clusters_users["non_user"]), 2)

clusters_users

In [ ]:
device_type = devices_user[["client_id", "desktop", "mobile", "tablet"]]

In [ ]:
# preparing needed columns
##TODO make it into the function

clusters_pages = emb_short.drop(columns=["is_user", "hits", "hits_watches", "days_active", "hits_per_day", "watches_hits_ratio"])\
    .merge(pages_user, on="client_id", how="inner")
T_clusters_pages = clusters_pages.groupby("cluster").sum().T

clusters_events = emb_short.drop(columns=["is_user", "hits", "hits_watches", "days_active", "hits_per_day", "watches_hits_ratio"])\
    .merge(events_user, on="client_id", how="outer")
T_clusters_events = clusters_events.groupby("cluster").sum().T

clusters_device_type = emb_short.drop(columns=["is_user", "hits", "hits_watches", "days_active", "hits_per_day", "watches_hits_ratio"])\
    .merge(devices_user, on="client_id", how="outer")

columns_os_mask = (~clusters_device_type.columns.str.contains("Android"))\
                   & (~clusters_device_type.columns.str.contains("iOS"))\
                   & (~clusters_device_type.columns.str.contains("OS"))\
                   & (~clusters_device_type.columns.str.contains("FreeBSD"))\
                   & (~clusters_device_type.columns.str.contains("Linux"))\
                   & (~clusters_device_type.columns.str.contains("Macintosh"))\
                   & (~clusters_device_type.columns.str.contains("Windows"))\
                   & (~clusters_device_type.columns.str.contains("Tizen"))

T_clusters_device_type = clusters_device_type[["client_id", "cluster", "desktop", "mobile", "tablet"]]\
    .groupby("cluster").sum().T

T_clusters_device_os = clusters_device_type.loc[:, (clusters_device_type.columns.str.contains("cluster")) |\
    (~columns_os_mask\
    & (~clusters_device_type.columns.isin(["desktop", "mobile", "tablet", "os_unknown"])))]\
    .groupby("cluster").sum().T

T_clusters_device_brand = clusters_device_type.loc[:, columns_os_mask\
    & (~clusters_device_type.columns.isin(["desktop", "mobile", "tablet", "os_unknown"]))]\
    .groupby("cluster").sum().T

clusters_tags = emb_short.drop(columns=["is_user", "hits", "hits_watches", "days_active", "hits_per_day", "watches_hits_ratio"])\
    .merge(tags_user, on="client_id", how="outer")
T_clusters_tags = clusters_tags.groupby("cluster").sum().T

In [ ]:
purchases = clusters_events[["cluster", "purchase"]].copy()
purchases["was_purch"] = purchases["purchase"]
purchases.loc[purchases["was_purch"] != 0, "was_purch"] = 1
purchases

In [ ]:
purchases.groupby(["cluster"]).agg({"purchase": ["sum", "count"], "was_purch": "sum"})

In [ ]:
clusters_buying = T_clusters_events_ratio\
    .T[["add_payment_info", "add_shipping_info", "add_to_cart", "begin_checkout", "checkout_progress", "purchase"]]
clusters_buying

In [ ]:
clusters_buying_rat = clusters_buying[["purchase"]].copy()

for col in clusters_buying.columns:
    clusters_buying_rat[f"ratio_{col}"] = clusters_buying[col]/clusters_buying["add_to_cart"]
clusters_buying_rat = clusters_buying_rat.drop(columns=["purchase"])
clusters_buying_rat_s = clusters_buying_rat.T.sort_values(by="cluster_0_rat", ascending=False).T
clusters_buying_rat_s

In [ ]:
# renaming the columns
##TODO make it into the function

i = 0
for col in T_clusters_pages.columns:
    T_clusters_pages = T_clusters_pages.rename(columns={col: f"cluster_{i}"})
    i += 1

i = 0
for col in T_clusters_events.columns:
    T_clusters_events = T_clusters_events.rename(columns={col: f"cluster_{i}"})
    i += 1
    
i = 0
for col in T_clusters_device_type.columns:
    T_clusters_device_type = T_clusters_device_type.rename(columns={col: f"cluster_{i}"})
    i += 1

i = 0
for col in T_clusters_device_os.columns:
    T_clusters_device_os = T_clusters_device_os.rename(columns={col: f"cluster_{i}"})
    i += 1

i = 0
for col in T_clusters_device_brand.columns:
    T_clusters_device_brand = T_clusters_device_brand.rename(columns={col: f"cluster_{i}"})
    i += 1
    
i = 0
for col in T_clusters_tags.columns:
    T_clusters_tags = T_clusters_tags.rename(columns={col: f"cluster_{i}"})
    i += 1

In [ ]:
cl_size = list(emb_short.groupby(["cluster"]).client_id.count())
cl_size.append(emb_short.groupby(["cluster"]).client_id.count().sum())

In [ ]:
if "Total" in T_clusters_tags.columns:
    T_clusters_tags = T_clusters_tags.drop(columns="Total")
T_clusters_tags["Total"] = T_clusters_tags.sum(axis=1)
T_clusters_tags_ratio = T_clusters_tags.copy()
i = 0
for col in T_clusters_tags.columns:
    T_clusters_tags_ratio[f"{col}_rat"] = T_clusters_tags_ratio[col]/cl_size[i]
    T_clusters_tags_ratio = T_clusters_tags_ratio.drop(columns=[col])
    i += 1

In [ ]:
T_clusters_tags_ratio.sort_values(by="cluster_0_rat", ascending=False).head(30)

In [ ]:
if "Total" in T_clusters_events.columns:
    T_clusters_events = T_clusters_events.drop(columns="Total")
T_clusters_events["Total"] = T_clusters_events.sum(axis=1)
T_clusters_events_ratio = T_clusters_events.copy()
i = 0
for col in T_clusters_events.columns:
    T_clusters_events_ratio[f"{col}_rat"] = T_clusters_events_ratio[col]/cl_size[i]
    T_clusters_events_ratio = T_clusters_events_ratio.drop(columns=[col])
    i += 1

In [ ]:
T_clusters_events_ratio.sort_values(by="cluster_3_rat", ascending=False).head(30)

In [ ]:
if "Total" in T_clusters_pages.columns:
    T_clusters_pages = T_clusters_pages.drop(columns="Total")
T_clusters_pages["Total"] = T_clusters_pages.sum(axis=1)
T_clusters_pages_ratio = T_clusters_pages.copy()
i = 0
for col in T_clusters_pages.columns:
    T_clusters_pages_ratio[f"{col}_rat"] = T_clusters_pages_ratio[col]/cl_size[i]
    T_clusters_pages_ratio = T_clusters_pages_ratio.drop(columns=[col])
    i += 1

In [ ]:
T_clusters_pages_ratio.sort_values(by="cluster_0_rat", ascending=False).head(30)

In [ ]:
if "Total" in T_clusters_device_type.columns:
    T_clusters_device_type = T_clusters_device_type.drop(columns="Total")
T_clusters_device_type["Total"] = T_clusters_device_type.sum(axis=1)
T_clusters_device_type_ratio = T_clusters_device_type.copy()

for col in T_clusters_device_type.columns:
    T_clusters_device_type_ratio[f"{col}_rat"] = T_clusters_device_type_ratio[col]/T_clusters_device_type_ratio[col].sum()
    T_clusters_device_type_ratio = T_clusters_device_type_ratio.drop(columns=[col])
    

In [ ]:
T_clusters_device_type_ratio

In [ ]:
if "Total" in T_clusters_device_brand.columns:
    T_clusters_device_brand = T_clusters_device_brand.drop(columns="Total")
T_clusters_device_brand["Total"] = T_clusters_device_brand.sum(axis=1)
T_clusters_device_brand_ratio = T_clusters_device_brand.copy()
for col in T_clusters_device_brand.columns:
    T_clusters_device_brand_ratio[f"{col}_rat"] = T_clusters_device_brand_ratio[col]/T_clusters_device_brand_ratio[col].sum()
    T_clusters_device_brand_ratio = T_clusters_device_brand_ratio.drop(columns=[col])

In [ ]:
T_clusters_device_brand_ratio.sort_values(by="cluster_5_rat", ascending=False).head(30)

### RESEARCHING CLUSTER 5

In [ ]:
curious = emb_short[emb_short["cluster"] == 0].reset_index(drop=True)
curious

In [ ]:
watches_pages = pages_user.filter(regex="client_id|watch")

In [ ]:
curious_whatch_pages = curious.merge(watches_pages, on="client_id", how="inner")

In [ ]:
curious_whatch_pages

In [ ]:
curious_watch5 = curious_whatch_pages[["client_id", "is_user", "watch5-pro", "watch5", "watch5-strap"]]

In [ ]:
hits_user = curious[curious["is_user"] == 1]['hits']
hits_non = curious[curious["is_user"] == 0]['hits']
totals = curious['hits']

upper_clip = 200

sns.set_theme()
plt.figure(figsize=(12, 4), dpi=80)
_, bins, _ = plt.hist(hits_user.clip(upper=upper_clip), bins=50, alpha=0.5,\
                      label='Curious, logged', weights=np.ones(hits_user.shape[0])/hits_user.shape[0])
plt.hist(hits_non.clip(upper=upper_clip), bins=bins, alpha=0.5,\
         label='Curious, non-logged', weights=np.ones(hits_non.shape[0])/hits_non.shape[0])
# plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('% Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
activity_by_logged = curious.groupby(["is_user"])\
    .agg({"hits": np.median,\
          "hits_watches": np.median,\
          "days_active": np.median,\
          "hits_per_day": np.median,\
          "watches_hits_ratio": np.median})
activity_by_logged

In [ ]:
pages_activity = curious_whatch_pages.drop(columns=["client_id", "is_user"]).sum().reset_index()
pages_activity.columns = ["page", "hits"]
# pages_activity.sort_values(by="hits", ascending=False)

In [ ]:
pages_activity[(pages_activity["page"].str.contains("5")) & ~(pages_activity["page"].str.contains("pro"))]

In [ ]:
pages_activity[(pages_activity["page"].str.contains("4")) & ~(pages_activity["page"].str.contains("pro"))\
               & ~(pages_activity["page"].str.contains("5"))]

In [ ]:
watch4_hits = pages_activity[(pages_activity["page"].str.contains("4")) & ~(pages_activity["page"].str.contains("pro"))\
               & ~(pages_activity["page"].str.contains("5"))]["hits"].sum()

In [ ]:
watch5_hits = pages_activity[(pages_activity["page"].str.contains("5"))\
                             & ~(pages_activity["page"].str.contains("pro"))]["hits"].sum()

In [ ]:
pages_activity[(pages_activity["page"].str.contains("5")) & (pages_activity["page"].str.contains("pro"))]

In [ ]:
pages_activity[(pages_activity["page"].str.contains("4")) & (pages_activity["page"].str.contains("pro"))\
               & ~(pages_activity["page"].str.contains("5"))]

In [ ]:
watch5_pro_hits = pages_activity[(pages_activity["page"].str.contains("5"))\
                                 & (pages_activity["page"].str.contains("pro"))]["hits"].sum()

In [ ]:
decisive = emb_short[emb_short["cluster"] == 3].reset_index(drop=True)
decisive

In [ ]:
decisive_whatch_pages = decisive.merge(watches_pages, on="client_id", how="inner")

In [ ]:
hits_user = decisive[decisive["is_user"] == 1]['hits']
hits_non = decisive[decisive["is_user"] == 0]['hits']
totals = decisive['hits']

upper_clip = 200

sns.set_theme()
plt.figure(figsize=(12, 4), dpi=80)
_, bins, _ = plt.hist(hits_user.clip(upper=upper_clip), bins=50, alpha=0.5,\
                      label='decisive, logged', weights=np.ones(hits_user.shape[0])/hits_user.shape[0])
plt.hist(hits_non.clip(upper=upper_clip), bins=bins, alpha=0.5,\
         label='decisive, non-logged', weights=np.ones(hits_non.shape[0])/hits_non.shape[0])
# plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('% Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
def pages_act(df):
    """
    Making the total hits out of activities
    """
    pages_activity = df.drop(columns=["client_id", "is_user"]).sum().reset_index()
    pages_activity.columns = ["page", "hits"]
    pages_activity = pages_activity.sort_values(by="hits", ascending=False)
    
    return pages_activity




In [ ]:
def watch_type(pages_activity):
    
    watch4_activity = pages_activity[(pages_activity["page"].str.contains("4")) & ~(pages_activity["page"].str.contains("pro"))\
                   & ~(pages_activity["page"].str.contains("5"))\
                                     & (pages_activity["page"] != "galaxy-watch4")]
    watch5_activity = pages_activity[(pages_activity["page"].str.contains("5"))\
                                 & ~(pages_activity["page"].str.contains("pro"))\
                                    & (pages_activity["page"] != "galaxy-watch5")]
    watch5_pro_activity = pages_activity[(pages_activity["page"].str.contains("5"))\
                                     & (pages_activity["page"].str.contains("pro"))]
    
    watch4_hits = watch4_activity["hits"].sum()
    watch5_hits = watch5_activity["hits"].sum()
    watch5_pro_hits = watch5_pro_activity["hits"].sum()
    print(f"watch4 hits: {watch4_hits}")
    print(f"watch5 hits: {watch5_hits}")
    print(f"watch5 pro hits: {watch5_pro_hits}")
    
    return watch4_activity, watch5_activity, watch5_pro_activity

In [ ]:
pages_curious = pages_act(curious_whatch_pages)
cur_watch4_activity, cur_watch5_activity, cur_watch5_pro_activity = watch_type(pages_curious)

In [ ]:
pages_dicisive = pages_act(decisive_whatch_pages)
dic_watch4_activity, dic_watch5_activity, dic_watch5_pro_activity = watch_type(pages_dicisive)

In [ ]:
dic_watch4_activity

In [ ]:
bundle = emb_short[emb_short["cluster"] == 4].reset_index(drop=True)

In [ ]:
hits_user = bundle[bundle["is_user"] == 1]['hits']
hits_non = bundle[bundle["is_user"] == 0]['hits']
totals = bundle['hits']

upper_clip = 300

sns.set_theme()
plt.figure(figsize=(12, 4), dpi=80)
_, bins, _ = plt.hist(hits_user.clip(upper=upper_clip), bins=50, alpha=0.5,\
                      label='bundle, logged', weights=np.ones(hits_user.shape[0])/hits_user.shape[0])
plt.hist(hits_non.clip(upper=upper_clip), bins=bins, alpha=0.5,\
         label='bundle, non-logged', weights=np.ones(hits_non.shape[0])/hits_non.shape[0])
# plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('% Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
bundle_whatch_pages = bundle.merge(watches_pages, on="client_id", how="inner")
pages_bundle = pages_act(bundle_whatch_pages)
bun_watch4_activity, bun_watch5_activity, bun_watch5_pro_activity = watch_type(pages_bundle)

In [ ]:
besp = emb_short[emb_short["cluster"] == 2].reset_index(drop=True)
besp_whatch_pages = besp.merge(watches_pages, on="client_id", how="inner")
pages_besp = pages_act(besp_whatch_pages)
besp_watch4_activity, besp_watch5_activity, besp_watch5_pro_activity = watch_type(pages_besp)

In [ ]:
besp_watch4_activity

In [ ]:
besp_4 = besp_whatch_pages[["galaxy-watch4-classic"]]["galaxy-watch4-classic"]

In [ ]:
plt.hist(besp_4, bins=50, alpha=0.5,\
                      label='besp', weights=np.ones(besp_4.shape[0])/besp_4.shape[0])

In [ ]:
plt.boxplot(besp_4)
plt.show()

In [ ]:
besp_4.reset_index().sort_values(by="galaxy-watch4-classic")

In [ ]:
besp_watch4_activity

In [ ]:
hits_user = besp[besp["is_user"] == 1]['hits']
hits_non = besp[besp["is_user"] == 0]['hits']
totals = besp['hits']

upper_clip = 2000

sns.set_theme()
plt.figure(figsize=(12, 4), dpi=80)
_, bins, _ = plt.hist(hits_user.clip(upper=upper_clip), bins=50, alpha=0.5,\
                      label='bespokers, logged', weights=np.ones(hits_user.shape[0])/hits_user.shape[0])
plt.hist(hits_non.clip(upper=upper_clip), bins=bins, alpha=0.5,\
         label='bespokers, non-logged', weights=np.ones(hits_non.shape[0])/hits_non.shape[0])
# plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('% Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
loyal = emb_short[emb_short["cluster"] == 5].reset_index(drop=True)
loyal_whatch_pages = loyal.merge(watches_pages, on="client_id", how="inner")
pages_loyal = pages_act(loyal_whatch_pages)
loyal_watch4_activity, loyal_watch5_activity, loyal_watch5_pro_activity = watch_type(pages_loyal)

In [ ]:
hits_user = loyal[loyal["is_user"] == 1]['hits']
hits_non = loyal[loyal["is_user"] == 0]['hits']
totals = loyal['hits']

upper_clip = 2000

sns.set_theme()
plt.figure(figsize=(12, 4), dpi=80)
_, bins, _ = plt.hist(hits_user.clip(upper=upper_clip), bins=50, alpha=0.5,\
                      label='loyal, logged', weights=np.ones(hits_user.shape[0])/hits_user.shape[0])
plt.hist(hits_non.clip(upper=upper_clip), bins=bins, alpha=0.5,\
         label='loyal, non-logged', weights=np.ones(hits_non.shape[0])/hits_non.shape[0])
# plt.hist(totals.clip(upper=upper_clip), bins=bins, alpha=0.5, label='Total', weights=np.ones(totals.shape[0])/totals.shape[0])
plt.legend(loc='upper right', prop={'size': 15})
plt.xlabel('Number of transactions', fontsize=10)
plt.ylabel('% Number of client_ids', fontsize=10)
plt.title("Amount of transactions per period", fontsize=15)

plt.show()

In [ ]:
round(pages_loyal[pages_loyal["page"] == "watchstudio"].reset_index(drop=True)["hits"][0]/loyal.shape[0], 2)

In [ ]:
round(pages_besp[pages_besp["page"] == "watchstudio"].reset_index(drop=True)["hits"][0]/besp.shape[0], 2)

In [ ]:
round(pages_bundle[pages_bundle["page"] == "watchstudio"].reset_index(drop=True)["hits"][0]/bundle.shape[0], 2)

In [ ]:
round(pages_dicisive[pages_dicisive["page"] == "watchstudio"].reset_index(drop=True)["hits"][0]/decisive.shape[0], 2)

In [ ]:
round(pages_curious[pages_curious["page"] == "watchstudio"].reset_index(drop=True)["hits"][0]/curious.shape[0], 2)

In [ ]:
how-to-use-the-galaxy-watch

In [ ]:
def page_invest(page_name):
    print(f"loyal interacting with {page_name}", round(pages_loyal[pages_loyal["page"] == page_name].reset_index(drop=True)["hits"][0]/loyal.shape[0], 2))
    print(f"bespoke interacting with {page_name}", round(pages_besp[pages_besp["page"] == page_name].reset_index(drop=True)["hits"][0]/besp.shape[0], 2))
    print(f"bundle interacting with {page_name}", round(pages_bundle[pages_bundle["page"] == page_name].reset_index(drop=True)["hits"][0]/bundle.shape[0], 2))
    print(f"decisive interacting with {page_name}", round(pages_dicisive[pages_dicisive["page"] == page_name].reset_index(drop=True)["hits"][0]/decisive.shape[0], 2))
    print(f"curious interacting with {page_name}", round(pages_curious[pages_curious["page"] == page_name].reset_index(drop=True)["hits"][0]/curious.shape[0], 2))

In [ ]:
page_invest("how-to-use-the-galaxy-watch")

In [ ]:
def page_invest_part(page_name):
    print(f"loyal interacting with {page_name}", round(pages_loyal[pages_loyal["page"].str.contains(page_name)]\
                                                       ["hits"].sum()/loyal.shape[0], 2))
    print(f"bespoke interacting with {page_name}", round(pages_besp[pages_besp["page"].str.contains(page_name)]\
                                                         ["hits"].sum()/besp.shape[0], 2))
    print(f"bundle interacting with {page_name}", round(pages_bundle[pages_bundle["page"].str.contains(page_name)]\
                                                        ["hits"].sum()/bundle.shape[0], 2))
    print(f"decisive interacting with {page_name}", round(pages_dicisive[pages_dicisive["page"].str.contains(page_name)]\
                                                          ["hits"].sum()/decisive.shape[0], 2))
    print(f"curious interacting with {page_name}", round(pages_curious[pages_curious["page"].str.contains(page_name)]\
                                                         ["hits"].sum()/curious.shape[0], 2))

In [ ]:
page_invest_part("strap")

In [ ]:
def tags_act(df, df_tags):
    
    tags = df.merge(df_tags, on="client_id", how="inner")
    tags_activity = tags.drop(columns=["client_id", "is_user", 'hits',\
                                       'hits_watches', 'hits_per_day',\
                                       'days_active', 'cluster', "gnb:open",\
                                       "view more", "carousel:arrow:right", "login", "image click"]).sum().reset_index()
    tags_activity.columns = ["page", "hits"]
    tags_activity = tags_activity.sort_values(by="hits", ascending=False)
    
    return tags_activity

In [ ]:
loyal_tag = tags_act(loyal, tags_user)
besp_tag = tags_act(besp, tags_user)
bundle_tag = tags_act(bundle, tags_user)
decisive_tag = tags_act(decisive, tags_user)
curious_tag = tags_act(curious, tags_user)

In [ ]:
def tags_invest(tag_name):
    print(f"loyal interacting with {tag_name}", round(loyal_tag[loyal_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/loyal.shape[0], 2))
    print(f"bespoke interacting with {tag_name}", round(besp_tag[besp_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/besp.shape[0], 2))
    print(f"bundle interacting with {tag_name}", round(bundle_tag[bundle_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/bundle.shape[0], 2))
    print(f"decisive interacting with {tag_name}", round(decisive_tag[decisive_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/decisive.shape[0], 2))
    print(f"curious interacting with {tag_name}", round(curious_tag[curious_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/curious.shape[0], 2))

In [ ]:
def tags_invest_purch(tag_name):
    print(f"loyal interacting with {tag_name}", round(loyal_tag[loyal_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/20648, 2))
    print(f"bespoke interacting with {tag_name}", round(besp_tag[besp_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/865, 2))
    print(f"bundle interacting with {tag_name}", round(bundle_tag[bundle_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/1714, 2))
    print(f"decisive interacting with {tag_name}", round(decisive_tag[decisive_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/1367, 2))
    print(f"curious interacting with {tag_name}", round(curious_tag[curious_tag["page"] == tag_name].reset_index(drop=True)["hits"][0]/4895, 2))

In [ ]:
tags_invest_purch("pre-order")

In [ ]:
tags_invest("pre-order")

In [ ]:
tags_invest("compare")

In [ ]:
tags_invest("sort:price ascending")

In [ ]:
pages_bundle

In [ ]:
besp_tag.head(30)

In [ ]:
curious_tag[curious_tag["page"].str.contains("colour")]

In [ ]:
curious_loyal.head(30)

In [ ]:
tag_loyal.head(30)

In [ ]:
pages_besp